# Scrapping des entrées

In [4]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import gc

# Initialisation du webdriver pour ouvrir le site de Marmiton
def start_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')  # Mode sans tête pour économiser la mémoire
    driver = webdriver.Firefox(options=options)
    return driver

driver = start_driver()


In [6]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Entrées"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=entr%C3%A9es&page="
    total_pages = 67  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_recipes.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_recipes.csv'.")

if __name__ == "__main__":
    main()


Bouton 'J'accepte tout' cliqué avec succès !
Scraping page 1...
15 recettes trouvées sur https://www.marmiton.org/recettes/recherche.aspx?aqt=entr%C3%A9es&page=1
Scraping les détails pour : https://www.marmiton.org/recettes/recette_bonhomme-de-neige-entree-de-noel_64160.aspx
Données extraites :
Nom : Bonhomme de neige : entrée de Noël
Ingrédients : persil plat, 1 cuillère à café vinaigre, 1 bocal asperge, 1 bocal carotte râpées, , 6  oeufs de poule, , 6  oeufs de caille, , 1 bocal céleri rave râpé, , 1 boîte de 400 g macédoine de légumes ou 2 boîtes de 200 g de thon soit au naturel soit à l'huile selon les goûts ou 250 g de jambon coupé en dés ou 250 g de filet de poulet coupé en dés, , 2 cuillères à soupe mayonnaise (light, aux oeufs ou au citron selon les goûts), , 1  cornichons aigre-doux, , 6  câpres, , 1  salade laitue, , fromage selon votre inspiration)
Scraping les détails pour : https://www.marmiton.org/recettes/recette_entree-fraicheur_328013.aspx
Données extraites :
Nom : Ent

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Bouton 'J'accepte tout' cliqué avec succès !
Scraping page 66...
15 recettes trouvées sur https://www.marmiton.org/recettes/recherche.aspx?aqt=entr%C3%A9es&page=66
Scraping les détails pour : https://www.marmiton.org/recettes/recette_soupe-de-vermicelles-ultra-rapide_62107.aspx
Données extraites :
Nom : Soupe de vermicelles ultra rapide
Ingrédients : 1 noix beurre, 2 poignées vermicelles (ou autres petites pâtes), 1  oignon, , 1  cube de bouillon de poule
Scraping les détails pour : https://www.marmiton.org/recettes/recette_tartare-vegetarien-de-betteraves-rouges-au-fromage-de-chevre_31529.aspx
Données extraites :
Nom : Tartare végétarien de betteraves rouges au fromage de chèvre
Ingrédients : poivre moulu, sel fin, 1 cuillère à soupe vinaigre de xérès, goutte tabasco, feuille persil plat, 15 g persil ciselé, , 300 g betterave rouges (crues à faire cuire ou déjà cuites – au choix), , 2 bûchettes fromage de chèvre ou 1 bûche cendrée de fromage de chèvre sec, , 75 g mayonnaise, , 1  écha

# Scrapping des plats 

In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Entrées"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=plats&page="
    total_pages = 67  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_recipes_plats.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_recipes_plats.csv'.")

if __name__ == "__main__":
    main()


# Scrapping des desserts

In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Desserts"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=desserts&page="
    total_pages = 67  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_desserts.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_desserts.csv'.")

if __name__ == "__main__":
    main()


### Scrapping des apéritifs

In [ ]:
# Accepter les cookies
def accept_cookies(driver):
    try:
        driver.get("https://www.marmiton.org/")
        time.sleep(5)  # Attendre que la page charge complètement
        accept_cookies_button = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div[3]/button[3]/span")
        accept_cookies_button.click()
        print("Bouton 'J'accepte tout' cliqué avec succès !")
    except Exception as e:
        print(f"Erreur lors du clic sur 'J'accepte tout' : {e}")

accept_cookies(driver)

# Fonction pour scraper une page
def scrape_page(driver, url):
    recipes = []
    try:
        driver.get(url)
        time.sleep(5)  # Attendre que la page charge complètement

        # Vérifier que la page est correctement chargée
        if "recettes" not in driver.current_url:
            print(f"La page {url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return recipes

        # Trouver tous les éléments contenant les recettes
        recipe_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[4]/div/div/div/a')
        print(f"{len(recipe_elements)} recettes trouvées sur {url}")

        for recipe in recipe_elements:
            try:
                # Extraire l'URL de la recette
                recipe_url = recipe.get_attribute("href")
                recipes.append({"url": recipe_url})
            except Exception as e:
                print(f"Erreur lors de l'extraction d'une recette : {e}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'URL {url} : {e}")
    return recipes

# Fonction pour scraper les ingrédients d'une recette
def scrape_recipe_details(driver, recipe_url):
    try:
        driver.get(recipe_url)
        print(f"Scraping les détails pour : {recipe_url}")

        # Vérifier que la page de recette est correctement chargée
        if "recette" not in driver.current_url:
            print(f"La recette {recipe_url} n'a pas été chargée correctement. URL actuelle : {driver.current_url}")
            return "", ""

        # Extraire le nom de la recette
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/h1'))
            ).text.strip()
        except Exception as e:
            print(f"Erreur lors de l'extraction du titre : {e}")
            title = ""

        # Cliquer sur "Voir plus" pour charger tous les ingrédients
        try:
            while True:
                voir_plus_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[3]/span[1]'))
                )
                voir_plus_button.click()
                time.sleep(2)
        except:
            pass

        # Extraire les ingrédients
        ingredients = []
        try:
            ingredient_blocks = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div[2]/div/div'))
            )
            for block in ingredient_blocks:
                try:
                    quantity = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="count"]') else ""
                    unit = block.find_element(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="card-ingredient-quantity"]/span[@class="unit"]') else ""
                    name = block.find_element(By.XPATH, './/span[@class="ingredient-name"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-name"]') else ""
                    complement = block.find_element(By.XPATH, './/span[@class="ingredient-complement"]').text.strip() if block.find_elements(By.XPATH, './/span[@class="ingredient-complement"]') else ""
                    ingredient = f"{quantity} {unit} {name} {complement}".strip()
                    ingredients.append(ingredient)
                except Exception as e:
                    print(f"Erreur lors de l'extraction d'un ingrédient : {e}")
        except Exception as e:
            print(f"Erreur lors de l'extraction des ingrédients : {e}")

        # Afficher les données extraites pour validation
        print("Données extraites :")
        print(f"Nom : {title}")
        print(f"Ingrédients : {', '.join(ingredients)}")

        # Si aucune donnée n'est extraite, retour immédiat
        if not title and not ingredients:
            print("Aucune donnée extraite pour cette recette, arrêt du processus.")
            return "", ""

        return title, ", ".join(ingredients)

    except Exception as e:
        print(f"Erreur lors du scraping des détails pour {recipe_url} : {e}")
        return "", ""

# Fonction principale pour le scraping
def main():
    global driver

    # URL de base pour les pages "Apéritifs"
    base_url = "https://www.marmiton.org/recettes/recherche.aspx?aqt=ap%C3%A9ritifs&page="
    total_pages = 24  # Nombre total de pages à scraper

    # Créer un fichier CSV
    fieldnames = ["Title", "URL", "Ingredients"]
    with open("marmiton_apéritifs.csv", "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        unique_urls = set()

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}...")
            page_url = f"{base_url}{page}"
            recipes = scrape_page(driver, page_url)

            for recipe in recipes:
                if recipe["url"] not in unique_urls:
                    unique_urls.add(recipe["url"])
                    title, ingredients = scrape_recipe_details(driver, recipe["url"])
                    writer.writerow({"Title": title, "URL": recipe["url"], "Ingredients": ingredients})

            print(f"Page {page} terminée. Total de recettes uniques extraites jusqu'à présent : {len(unique_urls)}")

            # Redémarrer le WebDriver tous les 5 pages pour éviter les erreurs de mémoire
            if page % 5 == 0:
                driver.quit()
                print("Redémarrage du navigateur pour libérer la mémoire...")
                driver = start_driver()
                accept_cookies(driver)

            time.sleep(5)  # Pause pour éviter d'être bloqué

    driver.quit()
    print("Scraping terminé. Résultats enregistrés dans 'marmiton_apéritifs.csv'.")

if __name__ == "__main__":
    main()


# Bases de données complétes des recettes

In [1]:
import pandas as pd

# Charger les fichiers CSV dans des DataFrames
file1= 'marmiton_recipes.csv'
file2= 'marmiton_recipes_plats.csv'
file3= 'marmiton_desserts.csv'
file4= 'marmiton_apéritifs.csv'

df1= pd.read_csv(file1)
df2= pd.read_csv(file2)
df3= pd.read_csv(file3)
df4= pd.read_csv(file4)

# Fusionner les deux DataFrames
# `ignore_index=True` réinitialise les index après la fusion
base_recettes= pd.concat([df1, df2, df3, df4], ignore_index=True)

# Supprimer les lignes avec des valeurs manquantes
base_recettes = base_recettes.dropna()

# Enregistrer le DataFrame fusionné dans un nouveau fichier CSV
output_file = "base_recettes.csv"
base_recettes.to_csv(output_file, index=False)

print(f"La fusion est terminée. Le fichier fusionné est enregistré sous le nom : {output_file}")


                


La fusion est terminée. Le fichier fusionné est enregistré sous le nom : base_recettes.csv
